In [ ]:
import pandas as pd
#вставить полный путь до csv файла с выгрузкой от клиента
s = input()
df = pd.read_csv(s, delimiter = '\t', low_memory=False)
df = df[ (df['UNIQ'] == 1) & (df['CAMPAIGN_MEDIUM']=='PKW') & (df['issued']==1)]
df['utm_content'] = df['COOKIE'].apply(lambda x: str(x)).apply(lambda x: x.split('=')[-1])
df['utm_campaign'] = df['COOKIE'].apply(lambda x: str(x)).apply(lambda x: x.split('=')[2:3])
df = df.filter(items = ['CAMPAIGN_SOURCE', 'CAMPAIGN_MEDIUM','issued','applied','UNIQ','LEAD_DATE','utm_content','utm_campaign'])
df.columns = ['utm_source','utm_medium','issued','applied','UNIQ','date','utm_content','utm_campaign']
df['utm_campaign'] = df['utm_campaign'].apply(lambda x: str(x)).apply(lambda x: x[2:(len(x)-9)])
df['utm_campaign'] = df['utm_campaign'].apply(lambda x: x.replace('|','_'))
df['utm_campaign_no_id'] = df['utm_campaign'].apply(lambda x: x.split('_')[1:]).apply(lambda x: '_'.join(x))

df['source_m'] = df.utm_campaign.apply(lambda x: 'YandexDirect' if 'yandex' in x else 'GoogleAdWords' if 'google' in x else 'NaN')                                             
df['campaign_type'] = df.utm_campaign.apply(lambda x: 'Search' if 'search' in x else 'Network' if 'network' in x else 'NaN')
df['month'] = df['date'].apply(lambda x: x.split('-')[1])
df.dropna()
df.reset_index()
writer = pd.ExcelWriter('SVC_EXPORT_'+s.split('/')[-1][:-4]+'.xlsx', engine='xlsxwriter')
df.to_excel(writer, 'Sheet1')
writer.save()
